<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Product_Marketing_AI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Marketing AI System


## Overview
This system helps create high-quality marketing images automatically. It takes in photos and optional audio or video, then processes, refines, and enhances them to produce beautiful marketing visuals for many different industries.

## Key Features

- **Easy Input:** Upload main and supplementary images, plus optional multimedia for extra context.
- **Smart Processing:** The system automatically cuts out key parts, improves image details, and boosts overall clarity.
- **Creative Prompts:** Custom prompts are generated to guide the image creation process, making it tailored to your needs.
- **Fast Generation:** Uses multiple AI models working together to generate and improve images quickly.
- **Quality Check:** Compares final images to the originals and provides simple quality feedback.
- **Simple Reports:** Automatically produces a brief report with the final prompt and quality scores.

## Benefits
- Saves time by automating the creation of professional marketing images.
- Provides consistent and attractive visuals optimized for your business.
- Easy to use with straightforward input and clear feedback.

Enjoy a seamless experience in making your marketing visuals stand out!

In [12]:
%pip install -q torch transformers diffusers opencv-python langchain langchain-huggingface tenacity numpy matplotlib base64 scikit-image
!pip install -qU langchain-google-genai groq together

ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 7.8 MB/s eta 0:00:00


In [13]:
import os
import cv2
import numpy as np
import base64
import json
import matplotlib.pyplot as plt
from PIL import Image
import io
import sys
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, StableDiffusionXLPipeline
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tenacity import retry, stop_after_attempt, wait_exponential
import time
import random
import warnings
warnings.filterwarnings("ignore")

# Together AI library for Blackforest Flux model inference
from together import Together

# Directories setup
print("Setting up directories...")
os.makedirs("outputs", exist_ok=True)
os.makedirs("uploads", exist_ok=True)
os.makedirs("cache", exist_ok=True)
OUTPUT_DIR = "outputs"
UPLOAD_DIR = "uploads"
CACHE_DIR = "cache"

# Determine execution environment
print("Determining execution environment...")
try:
    from google.colab import files
    ENV = "colab"
    print("Running in Colab environment.")
except ImportError:
    try:
        from IPython.display import FileUpload
        ENV = "jupyter"
        print("Running in Jupyter environment.")
    except ImportError:
        import tkinter as tk
        from tkinter import filedialog
        ENV = "standalone"
        print("Running in standalone environment.")

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# API keys (ensure these are set appropriately in your environment)
GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "your_groq_api_key")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY", "your_google_api_key")

Setting up directories...
Determining execution environment...
Running in Colab environment.
Using device: cuda


In [14]:
# Initialize models for Gemini and Stable Diffusion pipelines
def initialize_models():
    print("Initializing models...")
    model_status = {
        "sd_pipeline": None,
        "gemini_llm": None,
        "gemini_context": None,
        "sd_xl_pipeline": None
    }
    try:
        cache_path = os.path.join(CACHE_DIR, "controlnet_sd15")
        os.makedirs(cache_path, exist_ok=True)
        print("Loading ControlNet model...")
        controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/control_v11p_sd15_seg",
            torch_dtype=torch.float16,
            use_safetensors=True,
            cache_dir=cache_path
        ).to(device)

        print("Loading Stable Diffusion pipeline...")
        sd_pipeline = StableDiffusionControlNetPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=controlnet,
            torch_dtype=torch.float16,
            use_safetensors=True,
            cache_dir=cache_path
        ).to(device)

        if device == "cuda":
            sd_pipeline.enable_attention_slicing()
            if hasattr(torch, 'compile') and callable(getattr(torch, 'compile')):
                try:
                    print("Attempting torch.compile for the UNet...")
                    sd_pipeline.unet = torch.compile(sd_pipeline.unet, mode="reduce-overhead")
                except Exception as e:
                    print(f"Warning: Torch compile failed: {e}")

        model_status["sd_pipeline"] = sd_pipeline

        try:
            print("Loading Stable Diffusion XL pipeline (optional)...")
            sd_xl_pipeline = StableDiffusionXLPipeline.from_pretrained(
                "stabilityai/stable-diffusion-xl-base-1.0",
                torch_dtype=torch.float16,
                use_safetensors=True,
                variant="fp16",
                cache_dir=os.path.join(CACHE_DIR, "sdxl")
            ).to(device)
            if device == "cuda":
                sd_xl_pipeline.enable_attention_slicing()
            model_status["sd_xl_pipeline"] = sd_xl_pipeline
        except Exception as e:
            print(f"SDXL pipeline initialization failed (non-critical): {e}")
    except Exception as e:
        print(f"Error initializing Stable Diffusion pipeline: {e}")

    try:
        from langchain_google_genai import ChatGoogleGenerativeAI
        print("Initializing Gemini models...")
        gemini_llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp-image-generation",
            temperature=0.7,
            max_retries=3,
            google_api_key=GOOGLE_API_KEY
        )
        gemini_context = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",
            temperature=0,
            max_retries=3,
            google_api_key=GOOGLE_API_KEY
        )
        model_status["gemini_llm"] = gemini_llm
        model_status["gemini_context"] = gemini_context
    except Exception as e:
        print(f"Error initializing Gemini models: {e}")

    print("Model initialization complete.")
    return model_status

models = initialize_models()

Initializing models...
Loading ControlNet model...
Loading Stable Diffusion pipeline...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Attempting torch.compile for the UNet...
Loading Stable Diffusion XL pipeline (optional)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

SDXL pipeline initialization failed (non-critical): CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 2437 has 14.73 GiB memory in use. Of the allocated memory 14.37 GiB is allocated by PyTorch, and 274.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Initializing Gemini models...
Model initialization complete.
